# Hmac python



### Challenge Name: hmac_generate (/embsec/hmac_python/hmac_generate)


    The serial device will send you a variable-length frame of data. The data
    frame will begin with a short, little-endian integer size of the data to 
    follow. You must generate an HMAC-SHA256 signature using the key in 
    'hmackeyfile0.bin'.
    
    The data frame you will receive from the device will be formatted as follows:
    
    [ 0x2   ] [   variable...   ]
    ------------------------------
    | Size   |     Data          |
    ------------------------------
    
    You must send a 32-byte HMAC-SHA256 signature to the serial device.
    
    [     0x20     ]
    ----------------
    |  HMAC(Data)  |
    ----------------
    
    1. Read the HMAC key from 'hmackeyfile0.bin'
    2. Read the size of the data from the serial device
    3. Read the data from the serial device
    4. Generate and send a HMAC-SHA256 over the data
    5. Read the response
    



In [2]:
from embsec import Serial
import struct
from Crypto.Hash import HMAC, SHA256

def hmac_generate():
    ser = Serial("/embsec/hmac_python/hmac_generate")
    # Your code goes here!
    with open ('hmackeyfile0.bin', 'rb') as hmac:
        hmackey = hmac.read()
    data_size = struct.unpack("<h", ser.read(2))[0]
    data = ser.read(data_size)
    h = HMAC.new(hmackey, digestmod=SHA256)
    h.update(data)
    ser.write(h.digest())
    return ser.read_until()
hmac_generate()


b'embsec{hmac_generate_6a65fc57c238a1d9}\n'

### Challenge Name: hmac_verify (/embsec/hmac_python/hmac_verify)


    The serial device will send you a series of messages in the format
    described below. For each message you must check the attached signature 
    If verification fails, you must respond with a zero-byte '\x00'. If 
    verification passes, you must respond with a one-byte '\x01'. When the
    length of the message you are about to receive is zero , read a newline-
    terminated flag.

    The serial device will send signed messages in the following format:

    ----------------------------------------
    [ 0x2 ] [  Size bytes    ] [   0x20   ]
    ----------------------------------------
    | Size |     Data         | HMAC(Data) |
    ----------------------------------------



In [5]:
from embsec import Serial
import struct
from Crypto.Hash import HMAC, SHA256

def hmac_verify():
    ser = Serial("/embsec/hmac_python/hmac_verify")
    # Your code goes here!
    with open ('hmackeyfile1.bin', 'rb') as hmac:
        hmackey = hmac.read()
    while True:
        data_size = struct.unpack("<h", ser.read(2))[0]
        print('hi')
        if data_size == 0:
            print('break')
            break
        data = ser.read(data_size)
        hmac = ser.read(32)
        h = HMAC.new(hmackey, data, digestmod=SHA256)
        #this function does everything together
        #h.update(data) - hashes data (dont unclude data^)
        #you can do this multiple times with different blocks of data
        #.digest is the actual creation of the hmac
        if h.digest() == hmac:
            ser.write(struct.pack('1s', b'\x01'))
            print('good')
        else:
            ser.write(struct.pack('1s', b'\x00'))
            print('bad')
        
    print(h)
    print(hmac)
    return ser.read_until()

hmac_verify()


hi
good
hi
bad
hi
good
hi
good
hi
bad
hi
good
hi
good
hi
good
hi
break
bytearray(b'ib\xd5-\x9c\x92\x02C\xb9\xb1\x8a\x11\x86\xa8)\x99\xe3T\xe6:\x1d1\x93\xaf\x87n\xd1\x0f\x1e\xd0E2')


b'embsec{hmac_verify_fdf7a870fd0862fc}\n'